# Subword tokenization

**Проблема:**

* Для каждого языка - свой морфологический анализатор. Проблема для синтетических языков.
* Новые слова, имена, опечатки → **OOV (out-of-vocabulary)**.
* Частотное распределение слов — **длинный хвост** (закон Ципфа).
* Для нейронный сетей нужно ограничить размер словаря.

**Решение:**

* Разделить слова на **подслова (subwords)**.
* Получить компактный словарь и возможность собирать редкие слова.

--

## Пример

| Слово         | Частота | Обычная токенизация | Subword токенизация       |
| ------------- | ------- | ------------------- | ------------------------- |
| *голодный*    | высокая | `["голод"]`         | `["голод", "ный"]`        |
| *несчастный*  | низкая  | `["несчастн"]`      | `["не", "счаст", "ный"]`  |
| *мирззрение*  | редкая  |   (OOV)             | `["мир", "з", "зрение"]`  |



Можно сжать словарь, улучшить обобщение, сохранить смысловую структуру.

---

## Основная идея

Токенизация превращает текст в последовательность **единиц (токенов)**, которые:

* часто встречаются,
* могут комбинироваться для построения новых слов.


| Уровень     | Пример                 | Проблемы                           |
| ----------- | ---------------------- | ---------------------------------- |
| Символы     | `["p", "l", "a", "y"]` | слишком длинные последовательности |
| Слова       | `["play", "games"]`    | OOV, огромный словарь              |
| Части слов  | `["play", "ing"]`      | компромисс                         |

---

## Алгоритмы Subword Tokenization

### Byte Pair Encoding (BPE)

Идея:

* Начинаем с отдельных символов.
* Итеративно **сливаем самые частые пары** в единый токен.

Пример:

```
t h e r e -> t + h + e + r + e
```

сливаем частые пары: `t+h`, `th+e`, `the+r` и т.д.

Постепенно получаем: `the`, `there`, и т.д.

BPE строит словарь от символов до подсловных единиц.

---

### Пример BPE шагов

| Шаг | Частые пары | Новые токены |
| --- | ----------- | ------------ |
| 1   | `t`+`h`     | `th`         |
| 2   | `th`+`e`    | `the`        |
| 3   | `e`+`r`     | `er`         |
| 4   | `the`+`r`   | `ther`       |

---

### WordPiece

* Используется в **BERT**, **DistilBERT**, **ALBERT**.
* Похож на BPE, но критерий — **максимизация вероятности** слова по языковой модели, а не частоты пар.

Формально:
$$
\text{argmax}_V \sum_{w \in D} \log P(w | V)
$$

Преимущества:

* Более статистически обоснованный.
* Лучше работает на данных с разной морфологией.

---

### Unigram Language Model (SentencePiece)

* Используется в **SentencePiece**
* Начинает с **большого набора возможных субслов**, затем **удаляет** наименее вероятные.
* Основан на вероятностной модели:
  $$
  P(w) = \prod_i P(s_i)
  $$
  где $s_i$ — подслова.

Преимущества:

* Позволяет хранить несколько возможных разбиений.
* Гибче, чем BPE.

---

### Byte-level BPE (GPT-2, GPT-3)

* Работает на уровне **байтов**, а не символов Unicode.
* Универсален: поддерживает любой язык, даже эмодзи.
* Не требует нормализации.

Пример:

```
Text: "Привет!" → bytes → merges → subwords

```

---

## Сравнение алгоритмов

| Алгоритм           | Основа                     | Преимущества       | 
| ------------------ | -------------------------- | ------------------ | 
| **BPE**            | Частотность пар            | Прост, быстрый     | 
| **WordPiece**      | Максимизация вероятности   | Более точен        | 
| **Unigram**        | Модель вероятности подслов | Гибкий, устойчивый | 
| **Byte-level BPE** | Байты                     | Универсален        | 

---

## Популярные библиотеки

| Библиотека                  | Описание                            | Алгоритмы               |
| --------------------------- | ----------------------------------- | ----------------------- |
| **SentencePiece** (Google)  | Самая гибкая                        | Unigram, BPE            |
| **Hugging Face Tokenizers** | Быстрая реализация на Rust          | BPE, WordPiece, Unigram |

---


## Пример BPE 

Текст:

```
low lower lowest
```

Начало:

```
l o w _ l o w e r _ l o w e s t
```

1. Самая частая пара: `l o` → `lo`
2. Затем `lo w` → `low`
3. Потом `low e` → `lowe` и т.д.

Результат:

```
["low", "er", "est"]
```



# Использование Transformers

In [ ]:
import gzip
from typing import Iterable

# Чтение файла данных
def read_texts(fn: str="data/news.txt.gz") -> Iterable[str]:
    with gzip.open(fn, "rt", encoding="utf-8") as f:
        for line in f:
            yield line.strip().split("\t")[2]

In [ ]:
from tokenizers.pre_tokenizers import Whitespace
from tokenizers.trainers import WordPieceTrainer
from tokenizers.models import WordPiece
from tokenizers import normalizers
from tokenizers.normalizers import NFD, Lowercase, StripAccents
from tokenizers.decoders import WordPiece as WordPieceDecoder
from tokenizers import Tokenizer


s = "hello_world"


tokenizer = Tokenizer(WordPiece(unk_token="[UNK]"))
normalizer = normalizers.Sequence([NFD(), Lowercase(), StripAccents()])

tokenizer.pre_tokenizer = Whitespace()
tokenizer.normalizer = normalizer
tokenizer.decoder = WordPieceDecoder()

trainer = WordPieceTrainer(vocab_size=8000)

tokenizer.train_from_iterator(read_texts(), trainer=trainer)
tokenizer.save("data/news_tokenizer.json")

In [ ]:
tokenizer = Tokenizer.from_file("data/news_tokenizer.json")
res = tokenizer.encode("первому корове привет миру прививка")


res.ids

In [ ]:
import torch
import torch.nn as nn 

tokenizer.enable_padding(length=50)
tokenizer.encode("первому корове привет миру прививка")


res = tokenizer.encode_batch(["первому корове привет миру прививка", "хорошая погода"])
torch.tensor([x.ids for x in res])

In [ ]:
emb = nn.Embedding(10000, 100)

In [ ]:
# "hello world" -> [5, 7, 0, 0, 0] -> [[0.5, 0.2, ...], [0.7, 0.9, ....]]
res = tokenizer.encode_batch(["первому корове привет миру прививка при", "хорошая погода"])
input = torch.tensor([x.ids for x in res])
f = emb(input)
f